In [45]:
import mlflow
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [46]:
training_data=[(2,10),(2,6),(11,11),(6,9),(6,4),(1,2),(5,10),(4,9),(10,12),(7,5),(9,11),(4,6),(3,10),(3,8),(6,11)]


In [47]:
#Tracking

In [48]:
remote_server_tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(remote_server_tracking_uri)
experiment_name = 'ClusteringWithMlflowTut'
try:
    exp_id = mlflow.create_experiment(name=experiment_name)
except Exception as e:
    exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

In [49]:
range_of_k = range(2,6) 
for k in range_of_k :
    with mlflow.start_run(experiment_id=exp_id):
        untrained_model = KMeans(n_clusters=k)
        trained_model=untrained_model.fit(training_data)
        cluster_labels = trained_model.labels_
        score=silhouette_score(training_data, cluster_labels)
        mlflow.log_param('value_of_k', k)
        mlflow.log_metric('silhoutte_score', score)
        mlflow.sklearn.log_model(trained_model, "Clustering_Model")
        mlflow.end_run()

2022/09/01 20:56:01 WARNING mlflow.utils.requirements_utils: Found lz4 version (3.1.3+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==3.1.3' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/09/01 20:56:03 WARNING mlflow.utils.requirements_utils: Found lz4 version (3.1.3+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==3.1.3' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/09/01 20:56:05 WARNING mlflow.utils.requirements_utils: Found lz4 version (3.1.3+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==3.1.3' without the local version label to make it installable from PyP

In [ ]:
#Models

In [44]:
logged_model = 'runs:/761858718ed04c64866e783665c08c38/Clustering_Model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict([[1, 2], [3, 4], [5, 6]])

2022/09/01 18:07:32 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - typing-extensions (current: 4.3.0, required: typing-extensions==3.10.0.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


array([0, 4, 4], dtype=int32)

In [38]:
#Register a Model
model_name="MyModel"
model_version=2

In [39]:
from mlflow.tracking.client import MlflowClient
client=MlflowClient()
client.update_model_version(name=model_name,version=model_version, description="THis is a new model with enhanced features" )

<ModelVersion: creation_timestamp=1662032971423, current_stage='None', description='THis is a new model with enhanced features', last_updated_timestamp=1662035399399, name='MyModel', run_id='761858718ed04c64866e783665c08c38', run_link='', source='/home/aditya1117/codes/mlruns/1/761858718ed04c64866e783665c08c38/artifacts/Clustering_Model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [40]:
#change stage
client.transition_model_version_stage(name=model_name,version=model_version,stage="staging")

<ModelVersion: creation_timestamp=1662032971423, current_stage='Staging', description='THis is a new model with enhanced features', last_updated_timestamp=1662035486319, name='MyModel', run_id='761858718ed04c64866e783665c08c38', run_link='', source='/home/aditya1117/codes/mlruns/1/761858718ed04c64866e783665c08c38/artifacts/Clustering_Model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [41]:
#to production
client.transition_model_version_stage(name=model_name,version=model_version,stage="production")

<ModelVersion: creation_timestamp=1662032971423, current_stage='Production', description='THis is a new model with enhanced features', last_updated_timestamp=1662035553838, name='MyModel', run_id='761858718ed04c64866e783665c08c38', run_link='', source='/home/aditya1117/codes/mlruns/1/761858718ed04c64866e783665c08c38/artifacts/Clustering_Model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [ ]:
#Load Registered Model

In [23]:

model_uri="models:/MyModel/None"

In [24]:
import mlflow.pyfunc as pf
loaded_model= pf.load_model(model_uri)

In [26]:
loaded_model.predict([[1, 11], [3, 4], [5, 6]])

array([3, 4, 0], dtype=int32)

In [42]:
#delete model
client.delete_model_version(name=model_name,version=model_version,)